In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import dimension_reduction

In [2]:
data = pd.read_csv("/Users/sahana/Downloads/data.csv")

In [3]:
data.isna().sum()

SEQN                         0
Gender                       0
Age                          0
Marital_status               0
Pregnant                     0
                          ... 
Thiacloprid               1254
5-Hydroxyimidacloprid     1254
N-Desmethylacetamiprid    1254
glyphosate                   0
UACR                         0
Length: 75, dtype: int64

In [4]:
data.head()

,SEQN,Gender,Age,Marital_status,Pregnant,current_drinker,former_drinker_x,current_smoker,former_drinker_y,Aspirin_use,...,Dimethyldithiophosphate,Diethyldithiophosphate,Imidacloprid,Acetamiprid,Clothianidin,Thiacloprid,5-Hydroxyimidacloprid,N-Desmethylacetamiprid,glyphosate,UACR
0,83736,2,42,0,1,0.0,1.0,0.0,0.0,0.0,...,1,0,0.0,0.0,0.0,0.0,0.0,0.0,1,0
1,83739,1,4,0,0,NaN,NaN,NaN,NaN,NaN,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,0
2,83745,2,15,0,0,NaN,NaN,0.0,0.0,NaN,...,1,0,0.0,0.0,0.0,0.0,1.0,0.0,1,0
3,83746,2,4,0,0,NaN,NaN,NaN,NaN,NaN,...,1,0,NaN,NaN,NaN,NaN,NaN,NaN,0,0
4,83748,1,3,0,0,NaN,NaN,NaN,NaN,NaN,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,0


In [5]:
data.fillna(0, inplace=True)

In [6]:
data.drop(columns=['SEQN'],inplace=True)

In [7]:
data['Gender'].replace([1, 2], [0, 1], inplace=True)

In [8]:
health_condition = pd.get_dummies(data['Health_cond'], dtype=int, drop_first=True)
print(health_condition.head())

   1.0  2.0  3.0  4.0  5.0
0    0    0    0    1    0
1    0    0    0    0    0
2    0    0    1    0    0
3    0    0    0    0    0
4    0    0    0    0    0


In [9]:
data.drop(['Health_cond'], axis=1, inplace=True)
data = pd.concat([data, health_condition], axis =1)
data = data.rename(columns={1:'HC1', 2: 'HC2', 3: 'HC3', 4:'HC4',5:'HC5'}) # 4 binary columns enough to describe 5 categories

In [10]:
std_scaler = StandardScaler()
data[['Age']] = std_scaler.fit_transform(data[['Age']])

In [11]:
X = data.drop(columns=['UACR']) 

In [12]:
y = data['UACR']

In [13]:
X_significant = dimension_reduction.get_p_values_significant_features(X,y)

**************************************************
significant features
**************************************************
                         Feature_name       p-Value
0                            Diabetes  6.958695e-42
1                         Weak Kidney  3.112891e-35
2                        heart attack  3.395251e-22
3                        Kidney Stone  1.490515e-14
4            congestive heart failure  7.886976e-11
5                         Drug Addict  9.961026e-11
6                           emphysema  3.138472e-09
7          current_chronic bronchitis  7.786426e-08
8                                 Age  4.430869e-07
9                          Chest Cold  1.855921e-06
10                          arthritis  3.792928e-06
11                               gout  5.648975e-06
12                   former_drinker_x  9.732317e-05
13                             stroke  1.154268e-04
14                                HC2  1.269929e-04
15                         Chest_pain  1.3899

In [14]:
X_significant.head()

,Diabetes,Weak Kidney,heart attack,Kidney Stone,congestive heart failure,Drug Addict,emphysema,current_chronic bronchitis,Age,Chest Cold,...,days_hospitalised,Aspirin_use,thyroid,Gender,Oxypyrimidine,Farmer,High_Blood_pressure,Pneu_flu_ear infection,HC1,Diethylphosphate
0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,-0.108628,0.0,...,0.0,0.0,0.0,1,1,1.0,0.0,0.0,0,1
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.556004,0.0,...,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0,1
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.137026,0.0,...,0.0,0.0,0.0,1,1,0.0,0.0,0.0,0,1
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.556004,0.0,...,0.0,0.0,0.0,1,0,0.0,0.0,0.0,0,1
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.594093,0.0,...,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0,1


In [15]:
X_select = dimension_reduction.select_k_best(X,y,30)

**************************************************
selected features using mutual_info_classif
**************************************************
Index(['Age', 'current_drinker', 'former_drinker_y', 'High_Blood_pressure',
       'Chest_pain', 'stomach problem', 'Diabetes', 'Drug Addict',
       'days_hospitalised', 'Weak Kidney', 'Kidney Stone', 'Pesticide_used',
       'congestive heart failure', 'stroke', 'emphysema', 'chronic bronchitis',
       'kidney_cancer', 'smoked_in_pregnancy', 'Prescribed_medicine',
       '3,5,6-trichloropyridinol', '3-phenoxybenzoic acid', 'Oxypyrimidine',
       'para-Nitrophenol', 'Dichlorovnl-dimeth prop carboacid',
       'Diethylphosphate', 'Diethylthiophosphate', 'Acetamiprid',
       'N-Desmethylacetamiprid', 'HC2', 'HC4'],
      dtype='object')


In [16]:
X_select

,Age,current_drinker,former_drinker_y,High_Blood_pressure,Chest_pain,stomach problem,Diabetes,Drug Addict,days_hospitalised,Weak Kidney,...,3-phenoxybenzoic acid,Oxypyrimidine,para-Nitrophenol,Dichlorovnl-dimeth prop carboacid,Diethylphosphate,Diethylthiophosphate,Acetamiprid,N-Desmethylacetamiprid,HC2,HC4
0,-0.108628,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,1,1,1,1,1,1,0.0,0.0,0,1
1,-1.556004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1,0,1,0,1,0,0.0,0.0,0,0
2,-1.137026,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1,1,1,1,1,1,0.0,0.0,0,0
3,-1.556004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1,0,1,0,1,1,0.0,0.0,0,0
4,-1.594093,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1,0,1,0,1,0,0.0,0.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6216,0.310350,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,...,1,0,1,0,1,1,0.0,0.0,0,1
6217,0.310350,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,...,1,0,1,0,1,1,0.0,0.0,0,1
6218,-1.441737,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1,0,1,1,1,1,0.0,0.0,0,0
6219,-0.375250,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1,0,1,1,1,1,0.0,0.0,0,0


In [17]:
X_famd = dimension_reduction.famd_dim_reduction(X, 20)
X_famd

          eigenvalue % of variance % of variance (cumulative)
component                                                    
0             69.523        19.29%                     19.29%
1             22.129         6.14%                     25.42%
2             20.690         5.74%                     31.16%
3             17.278         4.79%                     35.96%
4             15.476         4.29%                     40.25%
5             14.810         4.11%                     44.36%
6             13.896         3.85%                     48.21%
7             13.367         3.71%                     51.92%
8             12.688         3.52%                     55.44%
9             12.585         3.49%                     58.93%
10            11.797         3.27%                     62.20%
11            11.436         3.17%                     65.37%
12            10.904         3.02%                     68.40%
13            10.352         2.87%                     71.27%
14      

component,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,-5.783580,0.724056,0.578823,2.361322,19.528641,4.818735,18.315548,6.548908,32.766092,69.863583,2.353071,32.856763,-0.985630,-1.443662,4.097973,-6.477170,3.781899,-3.188209,3.975819,0.818795
1,4.328968,6.051723,6.612760,-3.419720,0.420088,-0.733593,-0.162688,-0.207801,1.544654,-2.932603,-4.199705,1.523836,3.486231,0.353817,3.344944,-1.842585,-1.584721,-2.404044,3.416899,4.504747
2,-5.298410,5.549284,-2.268546,4.721722,5.370388,0.042535,-0.049910,-5.498218,-2.488485,2.287211,-1.596696,-2.472196,-2.212616,1.059135,-1.020217,4.013804,-1.260805,0.167003,2.397302,0.303442
3,-1.878658,7.169096,2.108791,-1.966623,2.689260,0.797960,0.146943,-0.754967,-2.770753,-0.296538,1.565359,0.553271,0.148806,0.413666,-5.601372,-2.542745,1.367733,-1.985703,1.190772,1.728900
4,0.891351,1.875329,5.976911,-2.600641,-0.089182,-0.174956,-0.455618,0.557641,0.440287,-1.885636,-2.283186,1.376643,1.268682,0.075020,1.727816,-4.966332,-2.212020,0.650955,0.635646,-3.322979
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6216,-3.083605,-6.179426,-3.306127,-2.075132,-2.776586,-3.124544,5.715568,3.136679,2.541650,-3.160586,-2.673664,2.765309,-2.209596,1.962418,-1.760293,-0.124988,9.352727,3.835903,-2.284997,2.175772
6217,-3.083605,-6.179426,-3.306127,-2.075132,-2.776586,-3.124544,5.715568,3.136679,2.541650,-3.160586,-2.673664,2.765309,-2.209596,1.962418,-1.760293,-0.124988,9.352727,3.835903,-2.284997,2.175772
6218,-5.320778,6.918094,-5.729132,-0.493899,1.001157,-3.310197,-0.346826,0.141998,-1.768732,-1.914610,1.183093,5.873830,3.808247,-2.583477,0.692767,3.185398,-3.162884,-0.014326,-0.100006,0.083616
6219,-5.322877,0.253721,-8.287097,3.619853,-2.087411,-2.533593,4.183868,7.305482,0.165303,-3.162579,-1.413395,1.976712,4.945766,1.355396,0.060630,-3.325965,0.303671,-3.132411,-1.946677,-0.299815


In [18]:
X_rfe = dimension_reduction.dim_red_using_rfe(X, y, 20)

In [19]:
X_rfe

,Gender,Age,Marital_status,current_drinker,former_drinker_y,Aspirin_use,High_Blood_pressure,Chest_pain,Diabetes,days_hospitalised,Weak Kidney,overweight,"2,4-D",Oxypyrimidine,Dichlorovnl-dimeth prop carboacid,Diethylthiophosphate,Dimethyldithiophosphate,N-Desmethylacetamiprid,glyphosate,HC3
0,1,-0.108628,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1,1,1,1,1,0.0,1,0
1,0,-1.556004,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,0,0,0.0,0,0
2,1,-1.137026,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1,1,1,1,0.0,1,1
3,1,-1.556004,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,1,1,0.0,0,0
4,0,-1.594093,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,0,0,0.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6216,0,0.310350,1,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0,0,0,1,1,0.0,1,0
6217,0,0.310350,1,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0,0,0,1,1,0.0,1,0
6218,0,-1.441737,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,1,1,1,0.0,1,0
6219,0,-0.375250,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,1,1,1,0.0,0,0
